In [1]:
import pandas as pd
import numpy as np 
from sklearn.svm import SVC
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.naive_bayes import GaussianNB, CategoricalNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb


## import data

In [ ]:
train_path = max(glob.glob('/Users/ivankoh/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/1D/NUS Y3S1/BT4222/Data/train/*.csv'), key=os.path.getctime) 
print(train_path)
df = pd.read_csv(train_path)
X_train, y_train = df.drop('state', axis=1).to_numpy(), df['state'].to_numpy()

test_path = max(glob.glob('/Users/ivankoh/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/1D/NUS Y3S1/BT4222/Data/test/*.csv'), key=os.path.getctime) 
print(test_path)
test = pd.read_csv(test_path)

X_test, y_test = test.drop('state', axis=1).to_numpy(), test['state'].to_numpy()

## best classifiers 

In [2]:
#SVC 
svc_best = {'kernel': 'linear', 'gamma': 0.1, 'C': 0.001}
svc_clf = SVC(**svc_best)
svc_clf.fit(X_train, y_train)

#LOGISTIC REGRESSION
lr_clf = LogisticRegression(C=0.02071613012624668, penalty='l2', solver='newton-cg')
lr_clf.fit(X_train, y_train)

#RANDOM FOREST
rf_best = {'n_estimators': 150,
 'min_samples_split': 2,
 'min_samples_leaf': 4,
 'max_features': 'auto',
 'max_depth': 8,
 'bootstrap': False}
rf_clf = RandomForestClassifier(**rf_best)
rf_clf.fit(X_train, y_train)

#XGBOOST 
xgb_clf = xgb.XGBClassifier(learning_rate=0.02,
                                          max_depth=3,
                                          n_estimators=3000,
                                          subsample=0.5,
                                          colsample_bytree=0.25,
                                          eval_metric='auc',
                                          verbosity=1,
                                          use_label_encoder=False)
xgb_clf.fit(X_train, y_train, early_stopping_rounds=20)

y_pred_test = best_clf.predict(X_test)

## NB

In [ ]:
train_ohe_cat = X_train.iloc[:, 422:437]
X_train['category'] = train_ohe_cat.idxmax(1)
train_ohe_loc = X_train.iloc[:, 437:-1]
X_train['location'] = train_ohe_loc.idxmax(1)

train_ohe_cat = X_test.iloc[:, 422:437]
X_test['category'] = train_ohe_cat.idxmax(1)
train_ohe_loc = X_test.iloc[:, 437:-1]
X_test['location'] = train_ohe_loc.idxmax(1)

X_train = X_train.drop(X_train.iloc[:, 422:-2].columns, axis=1)
X_test = X_test.drop(X_test.iloc[:, 422:-2].columns, axis=1)

X_train1 = X_train.copy()
X_test1 = X_test.copy()
binary_cols = ['staff_pick', 'has_video']
train_numeric = X_train1.select_dtypes(include='number').drop(binary_cols, axis=1) 
test_numeric = X_test1.select_dtypes(include='number').drop(binary_cols, axis=1) 

for i, col in enumerate(train_numeric.columns):
    X_train1[col], val = pd.qcut(X_train1[col], 5, retbins = True, duplicates = 'drop')
    X_test1[col] = pd.cut(X_test1[col], bins = val, duplicates = 'drop', include_lowest=True)
    
enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value = np.nan)
X_train1_fitted = enc.fit_transform(X_train1)
X_test1_fitted = enc.transform(X_test1)
imp = SimpleImputer(strategy="most_frequent") # impute nan values
X_train1_fitted = imp.fit_transform(X_train1_fitted)
X_test1_fitted = imp.transform(X_test1_fitted)

X_train2 = X_train.copy()
X_test2 = X_test.copy()
categorical_cols = ['category', 'location']
train_binary = X_train2[binary_cols]
train_categorical = X_train2[categorical_cols]
test_binary = X_test2[binary_cols]
test_categorical = X_test2[categorical_cols]

In [ ]:


cnb_clf = CategoricalNB(alpha=1.873817422860387e-09)
cnb_clf.fit(X_train1_fitted, y_train)


gnb_clf = GaussianNB(var_smoothing=0.005336699231206307)
gnb_clf.fit(X_new_train, y_train)
y_train_pred = gnb_clf.predict(X_new_train)
